# **GAN is short form of Generative Adversarial Network** and a deep learning architecture. GAN consists of 2 parts, Discriminator and Generator.
 The Generator tries to creat fake images and fool the Discriminator, and Discriminator tries to distinguish the images and label them as fake(0) or real(1).

 This zero-sum game continuees until the Generator can no longer creat images which fools the Discriminator and the Discriminator cannot be fooled.

 There are different types of GAN Models but we are using DCGAN which is the short form of Deep Convolutional GAN.

# Step 1 | Importing libraries¶


In [ ]:
# tensorflow and keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D, Dropout, Reshape, Conv2DTranspose
from tensorflow.keras.models import Sequential
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from keras.metrics import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import os
import PIL
import time
from IPython import display

# Step 2 | Preparing data and showing some images


In [ ]:
root_path = "/kaggle/input/animefacedataset"
root_path = pathlib.Path(root_path)

In [ ]:
# prepraing data
batch_size = 128

data = keras.utils.image_dataset_from_directory(
    directory=root_path,
    label_mode=None,
    batch_size=batch_size,
    image_size=(64,64))

In [ ]:
data


In [ ]:
# let's see some images of the dataset
plt.figure(figsize=(8,5))
for images in data.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
# normalizing the input image to the range [-1, 1]
data = data.map(lambda d : ((d-127.5)/127.5))
data

# Step 3 | Building Discriminator
What is Discriminator ?

The Discriminator is a Neural Network model which tries to distinguish the real images from fake images(generated by Generator) and label them as fake(0) or real(1).

Notes :

1.The image size is (64,64), so the input_shape of first conv2d layer should be (64,64,3).

2.The output of Discriminator is either a 0(fake) or 1(real).

3.Using "same" as padding ensures us that the output dimension is not going to change.

4.In the Discriminator function, all activations should be "LeakyReLU", exept the last layer which should be "sigmoid"

5.The last layer is using "sigmoid" as activation function to create a binary output, which real images are labeled as 1 and the fake ones are labeled as 0.

The Discriminator downsamples the input shape.
First let's build Discriminator function

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Dropout, Flatten, Dense, LeakyReLU, Input

def Discriminator():
    discriminator = Sequential()
    discriminator.add(Input(shape=(64, 64, 3)))

    discriminator.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(BatchNormalization())
    discriminator.add(Dropout(0.2))

    discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(BatchNormalization())
    discriminator.add(Dropout(0.2))

    discriminator.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(BatchNormalization())
    discriminator.add(Dropout(0.2))

    discriminator.add(Flatten())
    discriminator.add(Dropout(0.2))
    discriminator.add(Dense(1, activation="sigmoid"))

    return discriminator

D_model = Discriminator()
D_model.summary()


In [ ]:
# optimizer
D_optm = Adam(1e-4)

In [ ]:
latent_dim = 100


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Reshape, BatchNormalization, Conv2DTranspose, Input, ReLU
import tensorflow as tf

latent_dim = 100  # Define your latent dimension

def Generator():
    generator = Sequential()
    generator.add(Input(shape=(latent_dim,)))  # Input is a latent vector
    generator.add(Dense(4 * 4 * 256, use_bias=False))
    generator.add(Reshape((4, 4, 256)))
    generator.add(BatchNormalization())

    generator.add(Conv2DTranspose(128, kernel_size=(3, 3), strides=(2, 2), padding="same"))
    generator.add(ReLU())
    generator.add(BatchNormalization())

    generator.add(Conv2DTranspose(128, kernel_size=(3, 3), strides=(2, 2), padding="same"))
    generator.add(ReLU())
    generator.add(BatchNormalization())

    generator.add(Conv2DTranspose(128, kernel_size=(3, 3), strides=(2, 2), padding="same"))
    generator.add(ReLU())
    generator.add(BatchNormalization())

    generator.add(Conv2DTranspose(3, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="tanh"))

    return generator

# Create and view summary
G_model = Generator()
G_model.summary()


In [ ]:
# optimizer
G_optm = Adam(1e-4)

In [ ]:
# creating random noise
random_noise = tf.random.normal([1,latent_dim])

In [ ]:
# feeding random noise to Genereator
G_output_on_random_noise = G_model(random_noise, training=False)

In [ ]:
# showing the image output of G_model
plt.imshow(G_output_on_random_noise[0, :, :, 0])
plt.axis("off")

In [ ]:
# feeding the output of Generator to Discriminator
D_output_on_random_noise = D_model(G_output_on_random_noise)
print(D_output_on_random_noise)

In [ ]:
# The code of this cell is from keras sample.
class GAN(tf.keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = tf.keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = tf.keras.metrics.Mean(name="g_loss")
    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        seed = tf.random.normal(shape=(batch_size, self.latent_dim))
        # Decode them to fake images
        generated_images = self.generator(seed)
        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)
        # Assemble labels discriminating real from fake images
        labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))
        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        # Sample random points in the latent space
        seed = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))
        
        # Train the generator (note that we should *not* update the weights of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(seed))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {"d_loss": self.d_loss_metric.result(), "g_loss": self.g_loss_metric.result()}

In [ ]:
class GANCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, generator, discriminator, save_dir="checkpoints"):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.best_g_loss = float('inf')
        self.best_d_loss = float('inf')
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        g_loss = logs["g_loss"]
        d_loss = logs["d_loss"]

        if g_loss < self.best_g_loss:
            self.best_g_loss = g_loss
            self.generator.save_weights(os.path.join(self.save_dir, "best_generator.weights.h5"))
            print(f"✅ Epoch {epoch}: Saved new best generator (g_loss: {g_loss:.4f})")

        if d_loss < self.best_d_loss:
            self.best_d_loss = d_loss
            self.discriminator.save_weights(os.path.join(self.save_dir, "best_discriminator.weights.h5"))
            print(f"✅ Epoch {epoch}: Saved new best discriminator (d_loss: {d_loss:.4f})")


In [ ]:
# loss function
loss_fn = tf.keras.losses.BinaryCrossentropy()

In [ ]:
model = GAN(discriminator=D_model, generator=G_model, latent_dim=latent_dim)
model.compile(d_optimizer=D_optm, g_optimizer=G_optm, loss_fn=loss_fn)
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Callbacks
checkpoint_cb = GANCheckpoint(generator=G_model, discriminator=D_model, save_dir="checkpoints")

# Training
history = model.fit(
    data,
    epochs=100,
    callbacks=[checkpoint_cb]
)


In [ ]:
# creating a random nosie to feed it to the trained Generator model
noise = tf.random.normal([32, 100])
# Generatine new images using the trained Generator model 
generated_images = G_model(noise, training=False)

In [ ]:
# converting the input image to the range [0, 255]
generated_images1 = (generated_images+127.5)*127.5

In [ ]:
plt.figure(figsize=(8, 5))
for i in range(16):
    ax = plt.subplot(4, 4, i+1)
    plt.imshow(generated_images1[i].numpy().astype("uint8"))
    plt.axis('off')

plt.show()